# Part I: Reserve Replication, Single Contract Basis

## Preliminary Analysis: Convergence Rates for 1-dimensional Projections of Feature Space

In [1]:
# Create a FFN for 1-dimensional projection inputs

def single_dim_FFN(data, targets,activation_fct_lst= ['relu', 'linear'], optimizers='adam', loss_measure='mse', metric='mae', batchsize= 100, epochs=10, validation_split=0.25, callbacks_list = None, fit_model =True):
    
    n_out = targets.shape[1]
    model = Sequential()
    model.add(Dense(units = 3*n_out, activation = activation_fct_lst[0],input_dim = 1 ))
    for i in range(1,len(activation_fct_lst)):
        model.add(Dense(units = n_out, activation = activation_fct_lst[i]))
        
    model.compile(optimizer=opt,loss=loss_type,metrics= [metric])

    if fit_model==True: 
        model_history = model.fit(data, targets,batchsize,epochs = epochs, callbacks=callbacks_list, validation_split= val_share,verbose = 0)
        return model, model_history
    else: 
        return model

#### Plots of Convergence Rates

In [16]:
# Illustrate the training histories of several models for 1-dimensional projections

def model_single_dim_FFN_plots(hist_age, hist_sum, hist_dur, hist_aoc, 
                               model_age, model_sum, model_dur, model_aoc,
                               data_age_test, data_sum_test, data_dur_test, data_aoc_test,
                              option_plot_loss = True, option_plot_log_loss = True,
                              option_plot_mae = True):

    
    if option_plot_loss == True:
        ## Plot 1
        plt.figure(1, figsize = (10,6))
        plt.subplot(221)
        plt.plot(hist_age['loss'])
        #plt.plot(hist_age['val_loss'])
        plt.title('Age')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Training Set', 'Validation Set'], loc='upper right')

        plt.subplot(222)
        plt.plot(hist_sum['loss'])
        #plt.plot(hist_sum['val_loss'])
        plt.title('Sum Ins.')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        #plt.legend(['Training Set', 'Validation Set'], loc='upper right')

        plt.subplot(223)
        plt.plot(hist_dur['loss'])
        #plt.plot(hist_dur['val_loss'])
        plt.title('Duration')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        #plt.legend(['Training Set', 'Validation Set'], loc='upper right')

        plt.subplot(224)
        plt.plot(hist_aoc['loss'])
        #plt.plot(hist_aoc['val_loss'])
        plt.title('Age of Contract')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        #plt.legend(['Training Set', 'Validation Set'], loc='upper right')

        plt.tight_layout()
    
    if option_plot_log_loss == True:
        ####### Plot 2
        fig, axis = plt.subplots(2,2, figsize = (10,6))
        fig. suptitle('Convergence Rates (resp. log(.)) of ANNs w.r.t. several predictors (see headings) on the Training Data (i.e. 7000 Contracts)')
        axis[0][0].plot(np.log(hist_age['loss']))
        axis[0][0].set_title('Age')
        axis[0][0].set_xlabel('Epoch')
        axis[0][0].set_ylabel('log(Loss)')
        axis[0][1].plot(np.log(hist_sum['loss']))
        axis[0][1].set_title('Sum Insured')
        axis[0][1].set_xlabel('Epoch')
        axis[0][1].set_ylabel('log(Loss)')
        axis[1][0].plot(np.log(hist_dur['loss']))
        axis[1][0].set_title('Duration')
        axis[1][0].set_xlabel('Epoch')
        axis[1][0].set_ylabel('log(Loss)')
        axis[1][1].plot(np.log(hist_aoc['loss']))
        axis[1][1].set_title('Age of Contract')
        axis[1][1].set_xlabel('Epoch')
        axis[1][1].set_ylabel('log(Loss)')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    if option_plot_mae == True:
        ###### Plot 3
        fig, axis = plt.subplots(2,2, figsize = (10,6))
        fig.suptitle('Convergence Rates of ANNs w.r.t. one Feature Variable (see headings). Respective residual Features considered equal for all Contracts')
        axis[0][0].plot(np.log(hist_age['mean_absolute_error']))
        axis[0][0].set_title('Age')
        axis[0][0].set_xlabel('Epoch')
        axis[0][0].set_ylabel('log(mae)')
        axis[0][1].plot(np.log(hist_sum['mean_absolute_error']))
        axis[0][1].set_title('Sum Insured')
        axis[0][1].set_xlabel('Epoch')
        axis[0][1].set_ylabel('log(mae)')
        axis[1][0].plot(np.log(hist_dur['mean_absolute_error']))
        axis[1][0].set_title('Duration')
        axis[1][0].set_xlabel('Epoch')
        axis[1][0].set_ylabel('log(mae)')
        axis[1][1].plot(np.log(hist_aoc['mean_absolute_error']))
        axis[1][1].set_title('Age of Contract')
        axis[1][1].set_xlabel('Epoch')
        axis[1][1].set_ylabel('log(mae)')
        fig.text(.5, .05, 'Loss is calculated as mean absolute error of each contracts time points, averaged on all Contracts (Training Set: 7000)', ha='center')
        plt.tight_layout(rect=[0.03, 0.07, 0.9, 0.95])

    

    ###### Plot 4
    #### Plots of visual Prediction Accuracy

    # Predictions
    age_pred = model_age.predict(data_age_test)
    sum_pred = model_sum.predict(data_sum_test)
    dur_pred = model_dur.predict(data_dur_test)
    aoc_pred = model_aoc.predict(data_aoc_test)
    #age_pred_cum = targets_pred.sum(axis = 0)

    # Poorest Predictions absolute
    age_pp = np.abs(age_pred-targets_age_test).sum(axis = 1).argmax()
    sum_pp = np.abs(sum_pred-targets_sum_test).sum(axis = 1).argmax()
    dur_pp = np.abs(dur_pred-targets_dur_test).sum(axis = 1).argmax()
    aoc_pp = np.abs(aoc_pred-targets_aoc_test).sum(axis = 1).argmax()

    # Poorest Predictions relative (to target reserve)
    age_pp_rel = (np.abs(age_pred-targets_age_test).sum(axis = 1)/targets_age_test.sum(axis = 1)).argmax()
    sum_pp_rel = (np.abs(sum_pred-targets_sum_test).sum(axis = 1)/targets_sum_test.sum(axis = 1)).argmax()
    dur_pp_rel = (np.abs(dur_pred-targets_dur_test).sum(axis = 1)/targets_dur_test.sum(axis = 1)).argmax()
    aoc_pp_rel = (np.abs(aoc_pred-targets_aoc_test).sum(axis = 1)/targets_aoc_test.sum(axis = 1)).argmax()

    # Single Contract Reserve Point of View

    ### Age
    fig1, axis1 = plt.subplots(2,2, figsize = (10,6))
    plt.suptitle('Reserve Profile of selected Contracts - Variable Feature: Age')
    fig1.text(.15, .05,'Top row: Random Contracts. Bottom Row: Worst Fit (relative to Target Reserve) and absolute worst Fit (r)')
    axis1[0][0].plot(age_pred[0,:], 'r',label='Prediction')
    axis1[0][0].plot(targets_age_test[0,:], 'g^', label = 'Target')
    axis1[0][0].set_title('Age '+ str(data_age_test[0]))
    axis1[0][0].legend()
    axis1[0][1].plot(age_pred[2,:], 'r',targets_age_test[2,:], 'g^')
    axis1[0][1].set_title('Age '+ str(data_age_test[2]))
    axis1[1][0].plot(age_pred[age_pp_rel,:], 'r',targets_age_test[age_pp_rel,:], 'g^')
    axis1[1][0].set_title('Age '+ str(data_age_test[age_pp_rel]))
    axis1[1][1].plot(age_pred[age_pp,:], 'r',targets_age_test[age_pp,:], 'g^')
    axis1[1][1].set_title('Age '+ str(data_age_test[age_pp]))

    ### Sum insured
    fig2, axis2 = plt.subplots(2,2, figsize = (10,6))
    plt.suptitle('Reserve Profile of selected Contracts - Variable Feature: Sum Insured')
    fig2.text(.15, .05,'Top row: Random Contracts. Bottom Row: Worst Fit (relative to Target Reserve) and absolute worst Fit (r)')
    axis2[0][0].plot(sum_pred[0,:], 'r',label='Prediction')
    axis2[0][0].plot(targets_sum_test[0,:], 'g^', label = 'Target')
    axis2[0][0].set_title('Sum insured '+ str(data_sum_test[0]))
    axis2[0][0].legend()
    axis2[0][1].plot(sum_pred[1,:], 'r',targets_sum_test[1,:], 'g^')
    axis2[0][1].set_title('Sum insured '+ str(data_sum_test[1]))
    axis2[1][0].plot(sum_pred[sum_pp_rel,:], 'r',targets_sum_test[sum_pp_rel,:], 'g^')
    axis2[1][0].set_title('Sum insured '+ str(data_sum_test[sum_pp_rel]))
    axis2[1][1].plot(sum_pred[sum_pp,:], 'r',targets_sum_test[sum_pp,:], 'g^')
    axis2[1][1].set_title('Sum insured '+ str(data_sum_test[sum_pp]))

    ### Duration
    fig3, axis3 = plt.subplots(1,2, figsize = (10,3))
    #plt.suptitle('Reserve Profile of selected Contracts - Variable Feature: Duration')
    axis3[0].plot(dur_pred[0,:], 'r',label='Prediction')
    axis3[0].plot(targets_dur_test[0,:], 'g^', label = 'Target')
    axis3[0].set_xlabel('Time, $t$', fontsize = 'large')
    axis3[0].set_ylabel('Policy Value ${}_t V_{40}$',fontsize = 'large')
    axis3[0].legend()
    #axis3[0][1].plot(dur_pred[1,:], 'r',targets_dur_test[1,:], 'g^')
    #axis3[1][0].plot(dur_pred[dur_pp_rel,:], 'r',targets_dur_test[dur_pp_rel,:], 'g^')
    axis3[1].plot(dur_pred[dur_pp,], 'r',targets_dur_test[dur_pp,], 'g^')
    axis3[1].set_xlabel('Time, $t$', fontsize = 'large')

    ### Age of Contract
    fig4, axis4 = plt.subplots(2,2, figsize = (10,6))
    plt.suptitle('Reserve Profile of selected Contracts - Variable Feature: Age of Contract')
    axis4[0][0].plot(aoc_pred[0,:], 'r',label='Prediction')
    axis4[0][0].plot(targets_aoc_test[0,:], 'g^', label = 'Target')
    axis4[0][0].legend()
    axis4[0][1].plot(aoc_pred[1,:], 'r',targets_aoc_test[1,:], 'g^')
    axis4[1][0].plot(aoc_pred[aoc_pp_rel,:], 'r',targets_aoc_test[aoc_pp_rel,:], 'g^')
    axis4[1][1].plot(aoc_pred[aoc_pp,], 'r',targets_aoc_test[aoc_pp,], 'g^')

    # Cummulative Reserve Point of View
    #plt.plot(range(n_h), targets_pred_cum, 'r-', range(n_h), targets_test_cum, 'g^')
    #plt.title('NN-approximation - cumulative reserve profile')
    #plt.legend(['prediction', 'target'])
    #plt.show()
    return (data_dur_test[0],data_dur_test[dur_pp])

In [ ]:
# Illustrate the shift in the range of the paramters of a network from before to after training

def visualize_distribution_parameters(w_init, w_final):
    
    n = len(w_init)
    if len(w_final)!=n:
        print('Input weight lists of different lengths!')
        pass
    
    fig, ax = plt.subplots(n,2, figsize = (12,8))
    
    # Top row of plot seperately
    ax[0][0].hist(w_init[0].flatten())
    ax[0][0].set_title('At initilization \n', fontsize = 'large')
    ax[0][0].set_xlabel('$W^{(1)}_{ij}$, $i=1,\ldots,L_1$, $j=1,\ldots,p$', fontsize = 'large')
    ax[0][0].set_ylabel('Frequency', fontsize = 'large')
    ax[0][1].hist(w_final[0].flatten())
    ax[0][1].set_title('After $600$ epochs of training \n', fontsize = 'large')
    ax[0][1].set_xlabel('$W^{(1)}_{ij}$, $i=1,\ldots,L_1$, $j=1,\ldots,p$', fontsize = 'large')
    
    for i in range(1,n):
        ax[i][0].hist(w_init[i].flatten(), bins = w_init[i].shape[0])
        ax[i][1].hist(w_final[i].flatten(), bins = w_final[i].shape[0])
        ax[i][0].set_ylabel('Frequency', fontsize = 'large')
        if i%2==0:
            if i==2:
                ax[i][0].set_xlabel('$W^{(2)}_{ij}$, $i=1,\ldots,L_2$, $j=1,\ldots,L_1$', fontsize = 'large')
                ax[i][1].set_xlabel('$W^{(2)}_{ij}$, $i=1,\ldots,L_2$, $j=1,\ldots,L_2$', fontsize = 'large')
            if i==4:
                ax[i][0].set_xlabel('$W^{(3)}_{ij}$, $i=1,\ldots,L_3$, $j=1,\ldots,L_2$', fontsize = 'large')
                ax[i][1].set_xlabel('$W^{(3)}_{ij}$, $i=1,\ldots,L_3$, $j=1,\ldots,L_2$', fontsize = 'large')
            if i==6:
                ax[i][0].set_xlabel('$W^{(4)}_{ij}$, $i=1,\ldots,L_4$, $j=1,\ldots,L_3$', fontsize = 'large')
                ax[i][1].set_xlabel('$W^{(4)}_{ij}$, $i=1,\ldots,L_4$, $j=1,\ldots,L_3$', fontsize = 'large')
        else:
            if i== 1:
                ax[i][0].set_xlabel('$b^{(1)}_i$, $i=1,\ldots,L_1$', fontsize = 'large')
                ax[i][1].set_xlabel('$b^{(1)}_i$, $i=1,\ldots,L_1$', fontsize = 'large')
            if i== 3:
                ax[i][0].set_xlabel('$b^{(2)}_i$, $i=1,\ldots,L_2$', fontsize = 'large')
                ax[i][1].set_xlabel('$b^{(2)}_i$, $i=1,\ldots,L_2$', fontsize = 'large')
            if i== 5:
                ax[i][0].set_xlabel('$b^{(3)}_i$, $i=1,\ldots,L_3$', fontsize = 'large')
                ax[i][1].set_xlabel('$b^{(3)}_i$, $i=1,\ldots,L_3$', fontsize = 'large')
    
    #ax[0][0].hist(w_init[0])
    #ax[0][0].set_title('Initialization of \n Weights, Layer 1')
    #ax[1][0].hist(w_init[1])
    #ax[1][0].set_title('Biases, Layer 1')
    #ax[2][0].hist(w_init[2])
    #ax[2][0].set_title('Weights, Layer 2')
    #ax[3][0].hist(w_init[3])
    #ax[3][0].set_title('Biases, Layer 2')
    #ax[0][1].hist(w_final[0])
    #ax[0][1].set_title('After Training Config. of \n Weights, Layer 1')
    #ax[1][1].hist(w_final[1])
    #ax[1][1].set_title('Biases, Layer 1')
    #ax[2][1].hist(w_final[2])
    #ax[2][1].set_title('Weights, Layer 2')
    #ax[3][1].hist(w_final[3])
    #ax[3][1].set_title('Bias, Layer 2')
    plt.tight_layout()
    plt.show()
    return